# Analisis de datos para estadísticas

## **Datos de consultas**
Subir el archivo "Registro de consultas de usuarios 2025 (respuestas).xls" sin modificar nada.

In [ ]:
import pandas as pd
from google.colab import files
import io

# --- NUEVA SECCIÓN: Análisis de Consultas ---
print("\n" + "="*50)
print("--- Análisis de Consultas por Mes ---")
print("="*50 + "\n")

# --- 1. Cargar el archivo de consultas ---
print("Por favor, sube el archivo de la hoja de cálculo de consultas (desde formulario de Google).")
uploaded_consultas = files.upload()

# Verificar si se subió algún archivo
if not uploaded_consultas:
    print("No se subió ningún archivo. Saliendo del análisis de consultas.")
else:
    # Obtener el nombre del archivo subido
    file_name_consultas = next(iter(uploaded_consultas))
    print(f"Archivo '{file_name_consultas}' subido exitosamente.")

    # --- 2. Leer la hoja de cálculo ---
    try:
        # Leer como Excel (.xls o .xlsx)
        df_consultas = pd.read_excel(io.BytesIO(uploaded_consultas[file_name_consultas]))

        # --- 3. Validar y preparar datos ---
        # Nombres exactos de las columnas según la descripción
        required_columns_consultas = [
            'Marca temporal',
            'Tipo de usuario',
            'Medio de consulta',
            'Sala de lectura',
            'Referencia',
            'Servicios de digitalización internos',
            'Venta de publicaciones y merchandising',
            'Uso de la colección'
            # 'Número de matrícula' no se incluye ya que no se usa en el análisis
        ]

        if all(col in df_consultas.columns for col in required_columns_consultas):
            # Seleccionar solo las columnas necesarias
            df_consultas = df_consultas[required_columns_consultas].copy()

            print("\n--- Datos de consultas leídos correctamente ---")
            # Opcional: Imprimir head, columnas, total de registros para verificación
            # print(f"Primeras 5 filas:\n{df_consultas.head()}\n")
            # print(f"Columnas: {df_consultas.columns.tolist()}\n")
            # print(f"Total de registros: {len(df_consultas)}\n")

            # Convertir 'Marca temporal' a formato datetime
            # errors='coerce' convertirá las fechas inválidas a NaT (Not a Time)
            df_consultas['Marca temporal'] = pd.to_datetime(df_consultas['Marca temporal'], errors='coerce')

            # Eliminar filas con fechas inválidas si es necesario (o se ignorarán en el filtro por mes)
            df_consultas.dropna(subset=['Marca temporal'], inplace=True)


            # Convertir columnas numéricas y manejar posibles errores o valores vacíos
            numeric_cols = ['Servicios de digitalización internos', 'Venta de publicaciones y merchandising']
            for col in numeric_cols:
                # Asegurarse de que la columna existe antes de procesarla
                if col in df_consultas.columns:
                    df_consultas[col] = pd.to_numeric(df_consultas[col], errors='coerce').fillna(0).astype(int) # Convertir a número, NaN a 0, y luego a entero
                else:
                    print(f"Advertencia: Columna numérica esperada '{col}' no encontrada.")


            # --- 4. Solicitar el mes a analizar al usuario ---
            while True:
                try:
                    mes_analizar = int(input("Ingresa el número del mes que deseas analizar (1-12): "))
                    if 1 <= mes_analizar <= 12:
                        break # Salir del bucle si el mes es válido
                    else:
                        print("Número de mes inválido. Por favor, ingresa un número entre 1 y 12.")
                except ValueError:
                    print("Entrada no válida. Por favor, ingresa un número.")

            print(f"\nAnalizando datos para el mes: {mes_analizar}")

            # --- 5. Filtrar datos por el mes seleccionado ---
            df_mes = df_consultas[df_consultas['Marca temporal'].dt.month == mes_analizar].copy()

            if df_mes.empty:
                print(f"\nNo se encontraron registros para el mes {mes_analizar}.")
            else:
                print(f"\nSe encontraron {len(df_mes)} registros para el mes {mes_analizar}.\n")

                # --- 6. Generar la tabla de resultados ---
                print("--- Resultados del Análisis del Mes ---")

                # 6.1) Tipo de usuario: cantidad de ocurrencias de cada valor (sin incluir NaN)
                print("\nTipo de usuario:")
                # MODIFICADO: Eliminado dropna=False
                print(df_mes['Tipo de usuario'].value_counts())

                # 6.2) Medio de consulta: cantidad de ocurrencias de cada valor (sin incluir NaN)
                print("\nMedio de consulta:")
                 # MODIFICADO: Eliminado dropna=False
                print(df_mes['Medio de consulta'].value_counts())

                # 6.3) Sala de lectura: cantidad de ocurrencias de cada valor (sin incluir NaN)
                print("\nSala de lectura:")
                 # MODIFICADO: Eliminado dropna=False
                print(df_mes['Sala de lectura'].value_counts())

                # 6.4) Referencia: cantidad de ocurrencias de cada valor (sin incluir NaN)
                print("\nReferencia:")
                 # MODIFICADO: Eliminado dropna=False
                print(df_mes['Referencia'].value_counts())

                # 6.5) Servicios de digitalización internos: sumatoria de los datos
                print("\nTotal de Servicios de digitalización internos (horas):")
                print(df_mes['Servicios de digitalización internos'].sum())

                # 6.6) Venta de publicaciones y merchandising: sumatoria de los datos
                print("\nTotal de Venta de publicaciones y merchandising (ítems):")
                print(df_mes['Venta de publicaciones y merchandising'].sum())

                # 6.7) Uso de la colección: solo la cantidad de ocurrencias del valor “Reclamo”
                print("\nUso de la colección - Cantidad de 'Reclamo':")
                # Asegurarse de que la columna es string antes de comparar
                conteo_reclamos = df_mes[df_mes['Uso de la colección'].astype(str) == 'Reclamo'].shape[0]
                print(conteo_reclamos)

                print("\n" + "-" * 40) # Separador de fin de análisis del mes

            # Opcional: Restablecer opciones de visualización de pandas si se modificaron antes
            # pd.reset_option('display.max_rows')
            # pd.reset_option('display.max_columns')
            # pd.reset_option('display.expand_frame_repr')
            # pd.reset_option('display.max_colwidth')


        else:
            print("\nError: No se encontraron todas las columnas requeridas en el archivo de consultas.")
            print(f"Columnas esperadas: {required_columns_consultas}")
            print(f"Columnas encontradas: {df_consultas.columns.tolist()}")
            print("Por favor, verifica que los nombres de las columnas en tu archivo coincidan exactamente.")


    except FileNotFoundError:
        print(f"Error: El archivo '{file_name_consultas}' no fue encontrado.")
    except Exception as e:
        print(f"Ocurrió un error al procesar el archivo de consultas: {e}")

## **Datos de Selección Informativa, Web y Redes Sociales**
Subir el archivo "DSI - Comunicación - Redes Sociales 2025 (respuestas).xls" sin modificar nada.

In [ ]:
import pandas as pd
from google.colab import files
import io

# --- NUEVA SECCIÓN: Análisis de Comunicaciones ---
print("\n" + "="*50)
print("--- Análisis de Comunicaciones por Mes ---")
print("="*50 + "\n")

# --- 1. Cargar el archivo de comunicaciones ---
print("Por favor, sube el archivo de la hoja de cálculo de comunicaciones (newsletters, posteos, etc.).")
uploaded_comunicaciones = files.upload()

# Verificar si se subió algún archivo
if not uploaded_comunicaciones:
    print("No se subió ningún archivo. Saliendo del análisis de comunicaciones.")
else:
    # Obtener el nombre del archivo subido
    file_name_comunicaciones = next(iter(uploaded_comunicaciones))
    print(f"Archivo '{file_name_comunicaciones}' subido exitosamente.")

    # --- 2. Leer la hoja de cálculo ---
    try:
        # Leer como Excel (.xls o .xlsx)
        df_comunicaciones = pd.read_excel(io.BytesIO(uploaded_comunicaciones[file_name_comunicaciones]))

        # --- 3. Validar y preparar datos ---
        # Nombres exactos de las columnas según la descripción
        required_columns_comunicaciones = [
            'Marca temporal',
            'Cantidad de correos enviados',
            'Noticias',
            'Boletín',
            'Sitio web',
            'Posteos en Facebook',
            'Posteos en Instagram',
            'Posteos en Twitter',
            'Posteos en YouTube',
            'Posteos en Issuu'
        ]

        if all(col in df_comunicaciones.columns for col in required_columns_comunicaciones):
            # Seleccionar solo las columnas necesarias
            df_comunicaciones = df_comunicaciones[required_columns_comunicaciones].copy()

            print("\n--- Datos de comunicaciones leídos correctamente ---")
            # Opcional: Imprimir head, columnas, total de registros para verificación
            # print(f"Primeras 5 filas:\n{df_comunicaciones.head()}\n")
            # print(f"Columnas: {df_comunicaciones.columns.tolist()}\n")
            # print(f"Total de registros: {len(df_comunicaciones)}\n")

            # Convertir 'Marca temporal' a formato datetime
            df_comunicaciones['Marca temporal'] = pd.to_datetime(df_comunicaciones['Marca temporal'], errors='coerce')

            # Eliminar filas con fechas inválidas si es necesario (o se ignorarán en el filtro por mes)
            df_comunicaciones.dropna(subset=['Marca temporal'], inplace=True)

            # Identificar columnas a sumar (todas excepto 'Marca temporal')
            columns_to_sum = [col for col in required_columns_comunicaciones if col != 'Marca temporal']

            # Convertir columnas a sumar a numérico, tratar errores como 0
            for col in columns_to_sum:
                # Asegurarse de que la columna existe antes de procesarla (ya validado, pero extra seguro)
                if col in df_comunicaciones.columns:
                     df_comunicaciones[col] = pd.to_numeric(df_comunicaciones[col], errors='coerce').fillna(0).astype(int)
                else:
                     print(f"Advertencia: Columna esperada '{col}' no encontrada para sumar.")


            # --- 4. Solicitar el mes a analizar al usuario ---
            while True:
                try:
                    mes_analizar_com = int(input("Ingresa el número del mes que deseas analizar (1-12): "))
                    if 1 <= mes_analizar_com <= 12:
                        break # Salir del bucle si el mes es válido
                    else:
                        print("Número de mes inválido. Por favor, ingresa un número entre 1 y 12.")
                except ValueError:
                    print("Entrada no válida. Por favor, ingresa un número.")

            print(f"\nAnalizando datos para el mes: {mes_analizar_com}")

            # --- 5. Filtrar datos por el mes seleccionado ---
            df_com_mes = df_comunicaciones[df_comunicaciones['Marca temporal'].dt.month == mes_analizar_com].copy()

            if df_com_mes.empty:
                print(f"\nNo se encontraron registros para el mes {mes_analizar_com}.")
            else:
                print(f"\nSe encontraron {len(df_com_mes)} registros para el mes {mes_analizar_com}.\n")

                # --- 6. Calcular la sumatoria por columna ---
                # Seleccionar solo las columnas numéricas del DataFrame filtrado y sumar
                sumatorias_com = df_com_mes[columns_to_sum].sum()

                # --- 7. Generar la tabla de resultados ---
                # Convertir la Serie de sumatorias a un DataFrame
                tabla_sumarias_com = sumatorias_com.reset_index()
                tabla_sumarias_com.columns = ['Ítem', 'Total en el mes'] # Renombrar columnas

                # --- 8. Mostrar la tabla final ---
                print("--- Sumatoria de Comunicaciones por Ítem en el Mes ---")

                # Ajustar opciones de visualización si es necesario (aunque esta tabla es pequeña)
                pd.set_option('display.max_rows', None)
                pd.set_option('display.max_columns', None)
                pd.set_option('display.expand_frame_repr', False)
                pd.set_option('display.max_colwidth', None)

                print(tabla_sumarias_com)
                print("\n" + "-" * 40) # Separador de fin de análisis

            # Opcional: Restablecer opciones de visualización
            # pd.reset_option('display.max_rows')
            # pd.reset_option('display.max_columns')
            # pd.reset_option('display.expand_frame_repr')
            # pd.reset_option('display.max_colwidth')


        else:
            print("\nError: No se encontraron todas las columnas requeridas en el archivo de comunicaciones.")
            print(f"Columnas esperadas: {required_columns_comunicaciones}")
            print(f"Columnas encontradas: {df_comunicaciones.columns.tolist()}")
            print("Por favor, verifica que los nombres de las columnas en tu archivo coincidan exactamente.")


    except FileNotFoundError:
        print(f"Error: El archivo '{file_name_comunicaciones}' no fue encontrado.")
    except Exception as e:
        print(f"Ocurrió un error al procesar el archivo de comunicaciones: {e}")

## **Datos de circulación**
Eliminar del archivo “circulación.xls” todas las columnas excepto Destino, Nivel bibliográfico, Título y Autor.

In [ ]:
import pandas as pd
from google.colab import files
import io

# --- 1. Cargar el archivo ---
print("Por favor, sube el archivo de la hoja de cálculo de circulación.")
uploaded = files.upload()

# Verificar si se subió algún archivo
if not uploaded:
    print("No se subió ningún archivo. Saliendo del script.")
else:
    # Obtener el nombre del archivo subido (asumimos que solo se sube uno)
    file_name = next(iter(uploaded))
    print(f"Archivo '{file_name}' subido exitosamente.")

    # --- 2. Leer la hoja de cálculo ---
    try:
        # Intentar leer como Excel (.xlsx)
        if file_name.endswith('.xlsx'):
            df = pd.read_excel(io.BytesIO(uploaded[file_name]))
        # Intentar leer como CSV (.csv) si no es Excel
        elif file_name.endswith('.csv'):
             # Puedes ajustar el delimitador si es necesario (ej: sep=';')
            df = pd.read_csv(io.BytesIO(uploaded[file_name]))
        else:
            print("Formato de archivo no soportado. Por favor sube un archivo .xlsx o .csv")
            df = None # Asegurarse de que df sea None si el formato no es válido

        if df is not None:
            # --- 3. Renombrar columnas para facilitar el acceso (ajusta si tus nombres son ligeramente diferentes) ---
            # Intenta encontrar las columnas por nombre o por posición si sabes que siempre están en las mismas.
            # Aquí asumimos los nombres exactos proporcionados: 'Destino', 'Nivel bibliográfico', 'Título', 'Autor/es'
            # Puedes añadir lógica para encontrar columnas si los nombres varían ligeramente.
            required_columns = ['Destino', 'Nivel Bibliografico', 'Título', 'Autor Principal']
            if all(col in df.columns for col in required_columns):
                df = df[required_columns] # Seleccionar solo las columnas que necesitamos
                df.columns = ['Destino', 'Nivel_Bibliografico', 'Titulo', 'Autor_es'] # Renombrar a nombres más amigables para Python

                print("\n--- Datos leídos correctamente ---")
                print(f"Total de registros: {len(df)}\n")

                # --- 4. Realizar los análisis ---

                # 4.1) Sumatoria de los destinos de préstamo, discriminados por nivel bibliográfico.
                print("--- 1) Sumatoria de destinos de préstamo por nivel bibliográfico ---")
                # Usar pivot_table para una tabla resumen limpia
                tabla_destinos_por_nivel = df.pivot_table(
                    index='Destino',
                    columns='Nivel_Bibliografico',
                    aggfunc='size', # Cuenta el número de filas en cada grupo
                    fill_value=0    # Rellenar combinaciones sin datos con 0
                )
                # Asegurarse de que las columnas 'X' y 'M' existan aunque no haya datos para una de ellas
                for nivel in ['X', 'M']:
                    if nivel not in tabla_destinos_por_nivel.columns:
                         tabla_destinos_por_nivel[nivel] = 0

                # Reordenar columnas para que M y X (o viceversa) estén consistentemente
                column_order = [col for col in ['M', 'X'] if col in tabla_destinos_por_nivel.columns]
                other_cols = [col for col in tabla_destinos_por_nivel.columns if col not in column_order]
                tabla_destinos_por_nivel = tabla_destinos_por_nivel[column_order + other_cols]


                print(tabla_destinos_por_nivel)
                print("-" * 40) # Separador

                # 4.2) Para cada título de revista (nivel X) la cantidad de préstamos para destino.
                print("\n--- 2) Conteo de préstamos por destino para cada título de revista (Nivel X) ---")
                # Filtrar solo las filas donde Nivel_Bibliografico es 'X'
                df_revistas = df[df['Nivel_Bibliografico'] == 'X'].copy()

                if not df_revistas.empty:
                    # Agrupar por Título y Destino y contar
                    conteo_revistas_por_destino = df_revistas.groupby(['Titulo', 'Destino']).size().reset_index(name='Cantidad_Prestamos')

                    # --- AÑADIR ESTAS LÍNEAS ANTES DE IMPRIMIR LA TABLA ---
                    # Configurar opciones de visualización de pandas para mostrar todas las filas y columnas (si caben)
                    pd.set_option('display.max_rows', None)
                    pd.set_option('display.max_columns', None)
                    pd.set_option('display.expand_frame_repr', False) # Evitar que la tabla se envuelva si es ancha
                    pd.set_option('display.max_colwidth', None) # Evitar que el contenido de las celdas se trunque

                    # Mostrar la tabla plana (Título, Destino, Cantidad)
                    print(conteo_revistas_por_destino)

                    # --- Opcional: Restablecer las opciones por defecto después de imprimir ---
                    # pd.reset_option('display.max_rows')
                    # pd.reset_option('display.max_columns')
                    # pd.reset_option('display.expand_frame_repr')
                    # pd.reset_option('display.max_colwidth')


                else:
                    print("No se encontraron registros con 'Nivel bibliográfico' = 'X'.")

                print("-" * 40) # Separador



            else:
                print("\nError: No se encontraron todas las columnas requeridas ('Destino', 'Nivel bibliográfico', 'Título', 'Autor/es') en el archivo.")
                print(f"Columnas encontradas: {df.columns.tolist()}")
                print("Por favor, verifica que los nombres de las columnas en tu archivo coincidan exactamente.")


    except FileNotFoundError:
        print(f"Error: El archivo '{file_name}' no fue encontrado (esto no debería ocurrir si se subió correctamente).")
    except Exception as e:
        print(f"Ocurrió un error al procesar el archivo: {e}")

## Temas consultados anual
Pasar los datos del txt a una planilla de cálculo, eliminar el texto y dejar las columnas Total y Materia.

In [ ]:
import pandas as pd
from google.colab import files
import io
import re # Necesario para limpiar la cadena CDU

# --- 0. Diccionario de Temas CDU ---
# Este diccionario debe estar incluido en el script
temas_CDU = {
    '34:711.5': 'Derecho de la Edificación',
    '331.45': 'Seguridad en el trabajo',
    '332.72': 'Mercado inmobiliario',
    '332.82': 'Política Habitacional',
    '341.233.1': 'Fideicomisos',
    '342.72': 'Derechos de los Ciudadanos',
    '347': 'Derecho Civil',
    '347.238.3': 'Propiedad Horizontal',
    '347.4': 'Contratos',
    '347.51': 'Responsabilidad Civil',
    '347.513': 'Daños',
    '347.56': 'Responsabilidad Profesional',
    '347.948': 'Peritajes',
    '349.443': 'Ejercicio profesional',
    '352': 'Administración Pública',
    '378:72': 'Enseñanza de la arquitectura',
    '504.61': 'Impacto Ambiental',
    '534.84': 'Acústica arquitectónica',
    '535.6': 'Color',
    '556.18': 'Recursos Hídricos',
    '614.841.3': 'Instalaciones y seguridad c/incendio',
    '624': 'Ingeniería Civil',
    '624:69': 'Construcciones Civiles',
    '624.01': 'Estructuras',
    '624.012.45': 'Estructuras de hormigón',
    '624.014': 'Estructuras metálicas',
    '624.13': 'Geotecnia',
    '624.014.2': 'Estructuras de acero',
    '624.15': 'Cimentaciones',
    '65.012': 'Gerenciamiento de Proyectos',
    '65.012.4': 'Gestión de la calidad',
    '65.013': 'Dirección de obras',
    '657.31': 'Cómputos, costos y presupuestos',
    '657.47': 'Costos de obra',
    '657.92': 'Tasaciones',
    '658': 'Administración de empresas',
    '658.56': 'Control de Calidad',
    '658.8:72': 'Marketing para arquitectos',
    '681.3': 'Sistemas informáticos p/arq.',
    '69': 'Construcciones',
    '69-05': 'Organización de obras',
    '69.057': 'Construcciones prefabricadas',
    '69.059': 'Mantenimiento edilicio',
    '69.059.2': 'Patologías',
    '691.11': 'Madera',
    '691.32': 'Hormigón',
    '691.41': 'Construcción con adobe',
    '692.2': 'Medianería',
    '692.23': 'Fachadas',
    '692.4': 'Cubiertas',
    '692.6': 'Escaleras',
    '692.71': 'Parrillas',
    '696.1': 'Instalaciones sanitarias',
    '696.2': 'Instalaciones de Gas',
    '696.6': 'Instalaciones eléctricas',
    '697': 'Climatización',
    '7.01': 'Estética',
    '7.013': 'Feng Shui',
    '711:93': 'Historia urbana de las ciudades',
    '711.1': 'Planificación Urbana',
    '711.146': 'Espacio público',
    '711.16': 'Planes Estratégicos',
    '711.163': 'Proyectos urbanos',
    '711.16:504': 'Planes Urbanos Ambientales',
    '711.24': 'Planificación Territorial',
    '711.4': 'Planificación Urbana',
    '711.41': 'Diseño Urbano',
    '711.42': 'Gestión Urbana',
    '711.5': 'Barrios',
    '711.51': 'Códigos de Planeamiento',
    '711.58': 'Asentamientos',
    '711.6': 'Códigos de Edificación',
    '711.62': 'Códigos de Habilitaciones',
    '711.73': 'Vías de circulación',
    '711.8': 'Infraestructura de servicios',
    '711.95': 'Ecología urbana',
    '712': 'Paisajismo',
    '712.23': 'Áreas costeras',
    '719': 'Patrimonio urbano',
    '72:34': 'Arquitectura legal',
    '72(079)': 'Concursos',
    '72(82)': 'Arquitectura argentina',
    '72(821.1)': 'Arquitectura de Buenos Aires',
    '72-057': 'Biografías de Arquitectos',
    '72.01': 'Teoría arquitectónica',
    '72.011': 'Documentación de obra',
    '72.025': 'Patrimonio arquitectónico',
    '72.03': 'Estilos Arquitectónicos',
    '72.036': 'Arquitectura Moderna',
    '72.039': 'Arquitectura Contemporánea',
    '72.039.28': 'Arquitectura Sustentable',
    '72.057': 'Cocinas',
    '720': 'Estudios de Arquitectura',
    '725.23': 'Oficinas',
    '725.51': 'Arquitectura Hospitalaria',
    '725.54': 'Barreras Arquitectónicas', # Corregido (asumo Barreras)
    '725.58': 'Accesibilidad',
    '725.82': 'Teatros',
    '727.7': 'Museos',
    '728': 'Viviendas',
    '728.1': 'Vivienda Social',
    '728.2': 'Viviendas Multifamiliares',
    '728.3': 'Viviendas Unifamiliares',
    '741': 'Dibujo arquitectónico',
    '821': 'Literatura',
    '93': 'Historia'
}


# --- NUEVA SECCIÓN: Análisis de Temas Consultados por CDU ---
print("\n" + "="*50)
print("--- Análisis de Temas Consultados por CDU ---")
print("="*50 + "\n")

# --- 1. Cargar el archivo de temas consultados ---
print("Por favor, sube el archivo de la hoja de cálculo de temas consultados.")
uploaded_temas = files.upload()

# Verificar si se subió algún archivo
if not uploaded_temas:
    print("No se subió ningún archivo. Saliendo del análisis de temas.")
else:
    # Obtener el nombre del archivo subido
    file_name_temas = next(iter(uploaded_temas))
    print(f"Archivo '{file_name_temas}' subido exitosamente.")

    # --- 2. Leer la hoja de cálculo ---
    try:
        # Leer como Excel (.xls o .xlsx)
        df_temas = pd.read_excel(io.BytesIO(uploaded_temas[file_name_temas]))

        # --- 3. Validar y preparar datos ---
        required_columns_temas = ['TOTAL', 'MATERIA']
        if all(col in df_temas.columns for col in required_columns_temas):
            df_temas = df_temas[required_columns_temas].copy() # Seleccionar y copiar para evitar advertencias
            df_temas.columns = ['Total_Obras', 'CDU_Completa'] # Renombrar

            print("\n--- Datos de temas leídos correctamente ---")
            # Puedes dejar estas líneas de verificación si quieres, o borrarlas como dijimos antes
            # print(f"Primeras 5 filas:\n{df_temas.head()}\n")
            # print(f"Columnas: {df_temas.columns.tolist()}\n")
            # print(f"Total de registros: {len(df_temas)}\n")


            # Asegurarse de que la columna 'Total_Obras' sea numérica
            df_temas['Total_Obras'] = pd.to_numeric(df_temas['Total_Obras'], errors='coerce')
            # Eliminar filas donde 'Total_Obras' no pudo convertirse a número (NaN) o es NaN
            df_temas.dropna(subset=['Total_Obras'], inplace=True)
             # Convertir Total_Obras a entero si no tiene decimales (opcional, para que se vea más limpio)
            df_temas['Total_Obras'] = df_temas['Total_Obras'].astype(int)

            # Asegurarse de que la columna 'CDU_Completa' sea string para aplicar replace
            df_temas['CDU_Completa'] = df_temas['CDU_Completa'].astype(str)


            # --- 4. Limpiar la columna CDU (remover información desde el primer paréntesis en adelante) ---
            # Usamos una expresión regular para encontrar y reemplazar patrones como "(..." hasta el final
            df_temas['CDU_Principal'] = df_temas['CDU_Completa'].str.replace(r'\(.*$', '', regex=True).str.strip()

            # Eliminar filas donde la CDU principal quedó vacía después de la limpieza
            df_temas.dropna(subset=['CDU_Principal'], inplace=True)
            df_temas = df_temas[df_temas['CDU_Principal'] != ''].copy()

            # print("--- CDU limpiada ---")
            # print(f"Primeras 5 filas con CDU_Principal:\n{df_temas.head()}\n")


            # --- 5. Realizar la sumatoria por CDU principal ---
            # Agrupar por la CDU principal y sumar la columna 'Total_Obras'
            sumatoria_cdu = df_temas.groupby('CDU_Principal')['Total_Obras'].sum()

            # --- 6. Ordenar de mayor a menor ---
            sumatoria_cdu_sorted = sumatoria_cdu.sort_values(ascending=False)

            # --- 7. Seleccionar los 10 primeros puestos ---
            top_10_cdu = sumatoria_cdu_sorted.head(10)

            # --- 8. Generar la tabla final con nombres de tema ---
            # Convertir la Serie a DataFrame
            top_10_df = top_10_cdu.reset_index()
            top_10_df.columns = ['Número CDU', 'Cantidad de ocurrencias'] # Renombrar columnas

            # Mapear el número CDU al nombre del tema usando el diccionario
            top_10_df['Nombre del tema'] = top_10_df['Número CDU'].map(temas_CDU)

            # --- CORRECCIÓN: Evitar inplace=True para eliminar FutureWarnings ---
            top_10_df['Nombre del tema'] = top_10_df['Nombre del tema'].fillna("Tema no encontrado en diccionario")
            # --- FIN CORRECCIÓN ---

            # Reordenar las columnas a: Número CDU, Nombre del tema, Cantidad de ocurrencias
            final_table_temas = top_10_df[['Número CDU', 'Nombre del tema', 'Cantidad de ocurrencias']]


            # --- 9. Mostrar la tabla final ---
            print("\n--- 3) Top 10 Temas más Consultados por CDU ---")

            # Ajustar opciones de visualización
            pd.set_option('display.max_rows', None)
            pd.set_option('display.max_columns', None)
            pd.set_option('display.expand_frame_repr', False)
            pd.set_option('display.max_colwidth', None)

            print(final_table_temas)
            print("-" * 40) # Separador

            # Opcional: Restablecer opciones
            # pd.reset_option('display.max_rows')
            # pd.reset_option('display.max_columns')
            # pd.reset_option('display.expand_frame_repr')
            # pd.reset_option('display.max_colwidth')


        else:
            print("\nError: No se encontraron todas las columnas requeridas ('TOTAL', 'MATERIA') en el archivo de temas.")
            print(f"Columnas encontradas: {df_temas.columns.tolist()}")
            print("Por favor, verifica que los nombres de las columnas en tu archivo coincidan exactamente.")


    except FileNotFoundError:
        print(f"Error: El archivo '{file_name_temas}' no fue encontrado.")
    except Exception as e:
        print(f"Ocurrió un error al procesar el archivo de temas: {e}")